In [1]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
#Can't use keras on 129.114.32.33:8000
#from keras import layers, models, optimizers
import datetime, os
import statistics, tqdm

#to-do list
#1. Record the amount of time a classifier takes: start(timestamp) - end(timestamp), put it in .pkl.gz

#Observations:
# 1. RF gives highest accuracy, but takes a lot of time to train: 25 minutes and 15 minutes
# 2. Neural Network is weakest
# 3. NB gives satisfactory results within a minute.

trainDF = pd.concat([pd.read_pickle('../../data/2015/MasterData_2015.pkl.gz'),
                    pd.read_pickle('../../data/2014/MasterData_2014.pkl.gz'),
                    pd.read_pickle('../../data/2013/MasterData_2013.pkl.gz'),
                    pd.read_pickle('../../data/2012/MasterData_2012.pkl.gz')])
                                   
trainDF = trainDF[trainDF.TEXT.notna() & trainDF.NTEE.notna()]
trainDF['text'] = trainDF['TEXT'].astype(str)
trainDF['label'] = trainDF['NTEE'].astype(str)

counts = trainDF['label'].value_counts().sort_index().to_frame()
counts['category'] = counts.index
counts['train_sample']=(counts['label']/2).astype(int)


/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def dataformodel(trainDF):
    # split the dataset into training and validation datasets 
    train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

    # create a count vectorizer object 
    count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    count_vect.fit(trainDF['text'])

    # transform the training and validation data using count vectorizer object
    xtrain_count =  count_vect.transform(train_x)
    xvalid_count =  count_vect.transform(valid_x)  
    
    # word level tf-idf
    tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
    tfidf_vect.fit(trainDF['text'])
    xtrain_tfidf =  tfidf_vect.transform(train_x)
    xvalid_tfidf =  tfidf_vect.transform(valid_x)
    
    def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
        # fit the training dataset on the classifier
        classifier.fit(feature_vector_train, label)

        # predict the labels on validation dataset
        predictions = classifier.predict(feature_vector_valid)

        if is_neural_net:
            predictions = predictions.argmax(axis=-1)

        return metrics.accuracy_score(predictions, valid_y), metrics.precision_score(predictions, valid_y, average='weighted'), metrics.recall_score(predictions, valid_y, average='weighted')


    # Naive Bayes on Count Vectors
    acNB, prNB, rcNB = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
    
    # RF on Count Vectors
    acRF, prRF, rcRF = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
    
    acNN, prNN, rcNN = 0, 0, 0
    return [acNB, acRF, acNN, prNB, prRF, prNN, rcNB, rcRF, rcNN]
    
'''
for rec in counts.values:
    print('class & samplesize: ', rec[1], rec[2]) 
    
    train=trainDF.loc[trainDF['label']==rec[1]]
    accuracy, precision, recall = [[],[],[]], [[],[],[]], [[],[],[]]
    results = [[rec[1], rec[2]]]
    for iterate in range(0, 10):
        train = train.sample(n=rec[2])
        
        result=dataformodel(train)
        results.append(result)
        for j in range(0,3):
            accuracy[j].append(result[j])
            precision[j].append(result[j+3])
            recall[j].append(result[j+6])
            
    stats = [[], [], []]                                                                                                                                        
    for j in range(0,3):
        stats[0].append([statistics.mean(accuracy[j]), statistics.stdev(accuracy[j])])                                                                                                                                     
        stats[1].append([statistics.mean(precision[j]), statistics.stdev(precision[j])])                                                                                                                                             
        stats[2].append([statistics.mean(recall[j]), statistics.stdev(recall[j])])                                                                                                                                         
    
    stats = pd.DataFrame(stats)                                                                                                                                                             
    if(os.path.exists('../../data/results/classifier_stats_10.pkl.gz')):
        stats = pd.concat([pd.read_pickle('../../data/results/classifier_stats_10.pkl.gz'), stats]).drop_duplicates()
    
    stats.to_pickle('../../data/results/classifier_stats_10.pkl.gz')

    results = pd.DataFrame(results, columns=['acNB','acRF', 'acNN', 'prNB', 'prRF', 'prNN', 'rcNB', 'rcRF', 'rcNN'])
    if(os.path.exists('../../data/results/classifier_results_10.pkl.gz')):
        results = pd.concat([pd.read_pickle('../../data/results/classifier_results_10.pkl.gz'), results]).drop_duplicates()
    
    results.to_pickle('../../data/results/classifier_results_10.pkl.gz')
'''

"\nfor rec in counts.values:\n    print('class & samplesize: ', rec[1], rec[2]) \n    \n    train=trainDF.loc[trainDF['label']==rec[1]]\n    accuracy, precision, recall = [[],[],[]], [[],[],[]], [[],[],[]]\n    results = [[rec[1], rec[2]]]\n    for iterate in range(0, 10):\n        train = train.sample(n=rec[2])\n        \n        result=dataformodel(train)\n        results.append(result)\n        for j in range(0,3):\n            accuracy[j].append(result[j])\n            precision[j].append(result[j+3])\n            recall[j].append(result[j+6])\n            \n    stats = [[], [], []]                                                                                                                                        \n    for j in range(0,3):\n        stats[0].append([statistics.mean(accuracy[j]), statistics.stdev(accuracy[j])])                                                                                                                                     \n        stats[1].appen

In [3]:
accuracy, precision, recall = [[],[],[]], [[],[],[]], [[],[],[]]
results = []
for iterate in range(0,10): 
    
    train = pd.DataFrame()
    for rec in counts.values:
        train = pd.concat([train, trainDF.loc[trainDF['label']==rec[1]].sample(n=rec[2])])
        
    result=dataformodel(train)
    print(result)
    results.append(result)
    for j in range(0,3):
        accuracy[j].append(result[j])
        precision[j].append(result[j+3])
        recall[j].append(result[j+6])
            
stats = [[], [], []]                                                                                                                                        
for j in range(0,3):
    stats[0].append([statistics.mean(accuracy[j]), statistics.stdev(accuracy[j])])                                                                                                                                     
    stats[1].append([statistics.mean(precision[j]), statistics.stdev(precision[j])])                                                                                                                                             
    stats[2].append([statistics.mean(recall[j]), statistics.stdev(recall[j])])                                                                                                                                         
    
stats = pd.DataFrame(stats)                                                                                                                                                             
if(os.path.exists('../../data/results/classifier_stats_10.pkl.gz')):
    stats = pd.concat([pd.read_pickle('../../data/results/classifier_stats_10.pkl.gz'), stats]).drop_duplicates()
    
stats.to_pickle('../../data/results/classifier_stats_10.pkl.gz')

results = pd.DataFrame(results, columns=['acNB','acRF', 'acNN', 'prNB', 'prRF', 'prNN', 'rcNB', 'rcRF', 'rcNN'])
if(os.path.exists('../../data/results/classifier_results_10.pkl.gz')):
    results = pd.concat([pd.read_pickle('../../data/results/classifier_results_10.pkl.gz'), results]).drop_duplicates()
    
results.to_pickle('../../data/results/classifier_results_10.pkl.gz')

/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[0.6410256410256411, 0.7986880622863887, 0, 0.6828830315189934, 0.8123224353285377, 0, 0.6410256410256411, 0.7986880622863887, 0]


/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[0.6383895044982911, 0.799147853541159, 0, 0.6811789813199854, 0.8130791382853436, 0, 0.6383895044982911, 0.799147853541159, 0]


/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


...

[0.6438763468052171, 0.8012322405627845, 0, 0.6843477145260187, 0.8140909957328735, 0, 0.6438763468052171, 0.8012322405627845, 0]


/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[0.6416080432816834, 0.7960979048845158, 0, 0.6840611163836396, 0.8105175824395545, 0, 0.6416080432816834, 0.7960979048845158, 0]


/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[0.638328198997655, 0.8010636504360353, 0, 0.6800651457513969, 0.814535077747628, 0, 0.638328198997655, 0.8010636504360353, 0]
